In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from imblearn.over_sampling import (
    RandomOverSampler,
    SMOTE,
    ADASYN,
    BorderlineSMOTE,
    SVMSMOTE
)

In [24]:
# import kagglehub
# kagglehub.dataset_download("iammustafatz/diabetes-prediction-dataset")
# kagglehub.dataset_download("fedesoriano/stroke-prediction-dataset")
# kagglehub.dataset_download("l3llff/banana")
# kagglehub.dataset_download("gabrielsantello/cars-purchase-decision-dataset")
# kagglehub.dataset_download("youssefaboelwafa/hotel-booking-cancellation-prediction")
# kagglehub.dataset_download("shelvigarg/credit-card-buyers")
# kagglehub.dataset_download("shivamb/machine-predictive-maintenance-classification")
# kagglehub.dataset_download("fdemoribajolin/death-classification-icu")
# kagglehub.dataset_download("nareshbhat/wine-quality-binary-classification")
# kagglehub.dataset_download("mssmartypants/paris-housing-classification")
# kagglehub.dataset_download("rabieelkharoua/predict-online-course-engagement-dataset")
# kagglehub.dataset_download("amanalisiddiqui/fraud-detection-dataset")

In [2]:
banana = pd.read_csv("C:\\Users\\danis\\Downloads\\Datasets smote\\banana_quality.csv")

In [3]:
banana

,Size,Weight,Sweetness,Softness,HarvestTime,Ripeness,Acidity,Quality
0,-1.924968,0.468078,3.077832,-1.472177,0.294799,2.435570,0.271290,Good
1,-2.409751,0.486870,0.346921,-2.495099,-0.892213,2.067549,0.307325,Good
2,-0.357607,1.483176,1.568452,-2.645145,-0.647267,3.090643,1.427322,Good
3,-0.868524,1.566201,1.889605,-1.273761,-1.006278,1.873001,0.477862,Good
4,0.651825,1.319199,-0.022459,-1.209709,-1.430692,1.078345,2.812442,Good
...,...,...,...,...,...,...,...,...
7995,-6.414403,0.723565,1.134953,2.952763,0.297928,-0.156946,2.398091,Bad
7996,0.851143,-2.217875,-2.812175,0.489249,-1.323410,-2.316883,2.113136,Bad
7997,1.422722,-1.907665,-2.532364,0.964976,-0.562375,-1.834765,0.697361,Bad
7998,-2.131904,-2.742600,-1.008029,2.126946,-0.802632,-3.580266,0.423569,Bad


In [4]:
banana.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8000 entries, 0 to 7999
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Size         8000 non-null   float64
 1   Weight       8000 non-null   float64
 2   Sweetness    8000 non-null   float64
 3   Softness     8000 non-null   float64
 4   HarvestTime  8000 non-null   float64
 5   Ripeness     8000 non-null   float64
 6   Acidity      8000 non-null   float64
 7   Quality      8000 non-null   object 
dtypes: float64(7), object(1)
memory usage: 500.1+ KB


In [5]:
banana['Quality'].value_counts()

Quality
Good    4006
Bad     3994
Name: count, dtype: int64

In [6]:
from sklearn.preprocessing import LabelEncoder

In [7]:
label_encoder = LabelEncoder()
banana['Quality'] = label_encoder.fit_transform(banana['Quality'])

In [8]:
banana['Quality'].value_counts()

Quality
1    4006
0    3994
Name: count, dtype: int64

In [9]:
from imblearn.datasets import make_imbalance

In [10]:
from sklearn.model_selection import train_test_split
from collections import Counter

In [11]:
df_resampled, y_resampled = make_imbalance(banana, banana['Quality'], sampling_strategy={ 0 : 1200, 1: 4000},random_state=42)
df_resampled2 = df_resampled.copy()     # saving same data to be used for our proposed method 
print("Original class distribution:", Counter(y_resampled))
del df_resampled['Quality']

Original class distribution: Counter({1: 4000, 0: 1200})


In [12]:
def evaluate_oversampling(X, y, sampler, classifier):

    # Train-test split on resampled data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    X_train_resampled, y_train_resampled = sampler.fit_resample(X_train, y_train)
    print("Class distribution after SMOTE:", Counter(y_train_resampled))

    
    # Train the classifier
    classifier.fit(X_train_resampled, y_train_resampled)
    
    # Make predictions
    y_pred = classifier.predict(X_test)
    
    # Evaluate metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')
    
    return accuracy, precision, recall, f1, X_test, y_test

oversamplers = {
    "RandomOverSampler": RandomOverSampler(sampling_strategy='minority'),
    "SMOTE": SMOTE(sampling_strategy='minority'),
    "ADASYN": ADASYN(sampling_strategy='minority'),
    "BorderlineSMOTE": BorderlineSMOTE(sampling_strategy='minority'),
    "SVMSMOTE": SVMSMOTE(sampling_strategy='minority')
}
resultsNB = []
for name, sampler in oversamplers.items():
    accuracy, precision, recall, f1, xtest, ytest = evaluate_oversampling(df_resampled, y_resampled, sampler=sampler, classifier=GaussianNB())
    resultsNB.append({"Method": name, "Accuracy": accuracy, "Precision": precision, "Recall": recall, "F1 Score": f1})

resultsNB = pd.DataFrame(resultsNB)

# Print results
print(resultsNB)

Class distribution after SMOTE: Counter({1: 3216, 0: 3216})
Class distribution after SMOTE: Counter({1: 3216, 0: 3216})
Class distribution after SMOTE: Counter({0: 3257, 1: 3216})
Class distribution after SMOTE: Counter({1: 3216, 0: 3216})
Class distribution after SMOTE: Counter({1: 3216, 0: 3216})
              Method  Accuracy  Precision    Recall  F1 Score
0  RandomOverSampler  0.897115   0.908427  0.897115  0.900141
1              SMOTE  0.892308   0.903570  0.892308  0.895425
2             ADASYN  0.823077   0.870651  0.823077  0.833160
3    BorderlineSMOTE  0.804808   0.858978  0.804808  0.816402
4           SVMSMOTE  0.802885   0.864307  0.802885  0.815117


In [13]:
resultsKNN = []
for name, sampler in oversamplers.items():
    accuracy, precision, recall, f1, xtest, ytest = evaluate_oversampling(df_resampled, y_resampled, sampler=sampler, classifier=KNeighborsClassifier())
    resultsKNN.append({"Method": name, "Accuracy": accuracy, "Precision": precision, "Recall": recall, "F1 Score": f1})

resultsKNN = pd.DataFrame(resultsKNN)

# Print results
print(resultsKNN)

Class distribution after SMOTE: Counter({1: 3216, 0: 3216})
Class distribution after SMOTE: Counter({1: 3216, 0: 3216})
Class distribution after SMOTE: Counter({0: 3257, 1: 3216})
Class distribution after SMOTE: Counter({1: 3216, 0: 3216})
Class distribution after SMOTE: Counter({1: 3216, 0: 3216})
              Method  Accuracy  Precision    Recall  F1 Score
0  RandomOverSampler  0.968269   0.970623  0.968269  0.968751
1              SMOTE  0.971154   0.972534  0.971154  0.971476
2             ADASYN  0.956731   0.961519  0.956731  0.957666
3    BorderlineSMOTE  0.966346   0.968754  0.966346  0.966857
4           SVMSMOTE  0.964423   0.967504  0.964423  0.965041


In [14]:
resultsRF = []
for name, sampler in oversamplers.items():
    accuracy, precision, recall, f1, xtest, ytest = evaluate_oversampling(df_resampled, y_resampled, sampler=sampler, classifier=RandomForestClassifier())
    resultsRF.append({"Method": name, "Accuracy": accuracy, "Precision": precision, "Recall": recall, "F1 Score": f1})

resultsRF = pd.DataFrame(resultsRF)

# Print results
print(resultsRF)

Class distribution after SMOTE: Counter({1: 3216, 0: 3216})
Class distribution after SMOTE: Counter({1: 3216, 0: 3216})
Class distribution after SMOTE: Counter({0: 3257, 1: 3216})
Class distribution after SMOTE: Counter({1: 3216, 0: 3216})
Class distribution after SMOTE: Counter({1: 3216, 0: 3216})
              Method  Accuracy  Precision    Recall  F1 Score
0  RandomOverSampler  0.972115   0.972481  0.972115  0.972241
1              SMOTE  0.966346   0.966762  0.966346  0.966498
2             ADASYN  0.959615   0.961239  0.959615  0.960067
3    BorderlineSMOTE  0.962500   0.963299  0.962500  0.962762
4           SVMSMOTE  0.968269   0.969378  0.968269  0.968568


_______________________________________________________________________________________________________________________________________________________________________

In [38]:
banana[banana['Quality']==0].corr().abs().sum().sort_values()

Quality        0.000000
HarvestTime    1.788721
Weight         1.936141
Softness       2.108621
Acidity        2.109679
Sweetness      2.165802
Ripeness       2.195922
Size           2.436070
dtype: float64

In [26]:
df_resampled2[df_resampled2['Quality']==0].count()

Size           1200
Weight         1200
Sweetness      1200
Softness       1200
HarvestTime    1200
Ripeness       1200
Acidity        1200
Quality        1200
dtype: int64

In [27]:
3216-1200

2016

In [28]:
minority_samples = df_resampled2[df_resampled2['Quality']==0]
random_values = minority_samples['Size'].sample(n=2016, replace=True)
random_values = random_values.reset_index()
random_values = pd.DataFrame(random_values)
del random_values['index']
random_values

,Size
0,-1.969641
1,-4.255263
2,-3.583145
3,-0.632813
4,0.963569
...,...
2011,-4.556255
2012,-1.869040
2013,-2.622679
2014,0.318335


In [29]:
random_values['Weight'] = np.nan
random_values['Sweetness'] = np.nan
random_values['Softness'] = np.nan
random_values['HarvestTime'] = np.nan
random_values['Ripeness'] = np.nan
random_values['Acidity'] = np.nan
random_values['Quality'] = 0

In [30]:
# a.replace(np.nan, 'Pnan', inplace=True)
random_values

,Size,Weight,Sweetness,Softness,HarvestTime,Ripeness,Acidity,Quality
0,-1.969641,NaN,NaN,NaN,NaN,NaN,NaN,0
1,-4.255263,NaN,NaN,NaN,NaN,NaN,NaN,0
2,-3.583145,NaN,NaN,NaN,NaN,NaN,NaN,0
3,-0.632813,NaN,NaN,NaN,NaN,NaN,NaN,0
4,0.963569,NaN,NaN,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...
2011,-4.556255,NaN,NaN,NaN,NaN,NaN,NaN,0
2012,-1.869040,NaN,NaN,NaN,NaN,NaN,NaN,0
2013,-2.622679,NaN,NaN,NaN,NaN,NaN,NaN,0
2014,0.318335,NaN,NaN,NaN,NaN,NaN,NaN,0


In [32]:
def calculate_percentiles(nums):
    indexed_nums = [(num, i) for i, num in enumerate(nums)]
    sorted_nums = []
    for num_index in indexed_nums:
        inserted = False
        for i, sorted_num_index in enumerate(sorted_nums):
            if num_index[0] < sorted_num_index[0]:
                sorted_nums.insert(i, num_index)
                inserted = True
                break
        if not inserted:
            sorted_nums.append(num_index)
    length = len(sorted_nums)
    percentiles = [0] * length
    for i, num_index in enumerate(sorted_nums):
        original_index = num_index[1]
        percentile = ((i + 1) / length) * 100
        percentiles[original_index] = percentile
    return percentiles

def dataframe_to_percentiles(df):
    df_percentiles = df.copy()
    for column in df.columns:
        if pd.api.types.is_numeric_dtype(df[column]):
            df_percentiles[column] = calculate_percentiles(df[column].tolist())
    return df_percentiles


In [33]:
random_values['Size'] = calculate_percentiles(random_values['Size'])
random_values['Size'] = 'P' + random_values['Size'].round().astype(str,errors='ignore')

In [34]:
random_values

,Size,Weight,Sweetness,Softness,HarvestTime,Ripeness,Acidity,Quality
0,P41.0,NaN,NaN,NaN,NaN,NaN,NaN,0
1,P7.0,NaN,NaN,NaN,NaN,NaN,NaN,0
2,P14.0,NaN,NaN,NaN,NaN,NaN,NaN,0
3,P68.0,NaN,NaN,NaN,NaN,NaN,NaN,0
4,P92.0,NaN,NaN,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...
2011,P5.0,NaN,NaN,NaN,NaN,NaN,NaN,0
2012,P43.0,NaN,NaN,NaN,NaN,NaN,NaN,0
2013,P28.0,NaN,NaN,NaN,NaN,NaN,NaN,0
2014,P86.0,NaN,NaN,NaN,NaN,NaN,NaN,0


In [35]:
random_values['Size'].unique()

array(['P41.0', 'P7.0', 'P14.0', 'P68.0', 'P92.0', 'P97.0', 'P57.0',
       'P82.0', 'P66.0', 'P59.0', 'P18.0', 'P56.0', 'P32.0', 'P52.0',
       'P65.0', 'P89.0', 'P35.0', 'P34.0', 'P26.0', 'P42.0', 'P22.0',
       'P63.0', 'P93.0', 'P46.0', 'P76.0', 'P15.0', 'P79.0', 'P85.0',
       'P36.0', 'P55.0', 'P27.0', 'P60.0', 'P21.0', 'P83.0', 'P54.0',
       'P74.0', 'P64.0', 'P61.0', 'P50.0', 'P87.0', 'P43.0', 'P39.0',
       'P58.0', 'P33.0', 'P84.0', 'P3.0', 'P13.0', 'P90.0', 'P86.0',
       'P99.0', 'P19.0', 'P9.0', 'P73.0', 'P2.0', 'P48.0', 'P88.0',
       'P77.0', 'P81.0', 'P31.0', 'P38.0', 'P24.0', 'P94.0', 'P17.0',
       'P71.0', 'P12.0', 'P49.0', 'P78.0', 'P44.0', 'P62.0', 'P37.0',
       'P11.0', 'P98.0', 'P23.0', 'P69.0', 'P40.0', 'P70.0', 'P91.0',
       'P80.0', 'P6.0', 'P53.0', 'P1.0', 'P47.0', 'P51.0', 'P4.0',
       'P25.0', 'P95.0', 'P10.0', 'P30.0', 'P0.0', 'P96.0', 'P75.0',
       'P8.0', 'P45.0', 'P29.0', 'P16.0', 'P67.0', 'P5.0', 'P72.0',
       'P28.0', 'P100.0', 'P20

In [36]:
bananaP = dataframe_to_percentiles(minority_samples)
bananaP = 'P' + bananaP.round().astype(str,errors='ignore')
bananaP['Quality'] = 0

In [37]:
ZERO = pd.concat([bananaP,random_values], ignore_index=True)
ZERO['Quality']=0
ZERO

,Size,Weight,Sweetness,Softness,HarvestTime,Ripeness,Acidity,Quality
0,P48.0,P54.0,P21.0,P84.0,P33.0,P34.0,P67.0,0
1,P54.0,P50.0,P42.0,P37.0,P38.0,P90.0,P30.0,0
2,P27.0,P87.0,P40.0,P43.0,P21.0,P52.0,P9.0,0
3,P77.0,P35.0,P13.0,P21.0,P72.0,P96.0,P30.0,0
4,P85.0,P58.0,P5.0,P58.0,P30.0,P35.0,P76.0,0
...,...,...,...,...,...,...,...,...
3211,P5.0,NaN,NaN,NaN,NaN,NaN,NaN,0
3212,P43.0,NaN,NaN,NaN,NaN,NaN,NaN,0
3213,P28.0,NaN,NaN,NaN,NaN,NaN,NaN,0
3214,P86.0,NaN,NaN,NaN,NaN,NaN,NaN,0


In [38]:


# Step 1: Identify unique values of 'pH'
unique_size = bananaP['Size'].unique()

# Step 2: Create a dictionary to store non-missing values for each variable
# Initialize the dictionary
imputation_info = {}

# Iterate over each variable (excluding 'pH') that has missing values
for column in bananaP.columns:
    if column != 'Size' and ZERO[column].isna().sum() > 0:
        imputation_info[column] = {}
        
        # Iterate over each unique value of 'pH'
        for size_value in unique_size:
            # Get the non-missing values of the variable where 'pH' is equal to the current pH_value
            non_missing_values = bananaP.loc[bananaP['Size'] == size_value, column].dropna().values
            imputation_info[column][size_value] = non_missing_values

In [39]:
# import random
import statistics
def fill_missing_values2(row, imputation_info):
    # For each column, check if it has a missing value
    for column in imputation_info:
        if pd.isna(row[column]):
            size_value = row['Size']  # Get the corresponding pH value for the row
            if size_value in imputation_info[column]:
                possible_values = imputation_info[column][size_value]
                if len(possible_values) > 0:
                    row[column ] = statistics.mode(possible_values)
                    # row[column] = random.choice(possible_values)

    return row

myMode = ZERO.apply(lambda row: fill_missing_values2(row, imputation_info), axis=1)

In [40]:
# myMode[myMode.isna().any(axis=1)]
# myMode = myMode.fillna(myMode.iloc[0])
myMode

,Size,Weight,Sweetness,Softness,HarvestTime,Ripeness,Acidity,Quality
0,P48.0,P54.0,P21.0,P84.0,P33.0,P34.0,P67.0,0
1,P54.0,P50.0,P42.0,P37.0,P38.0,P90.0,P30.0,0
2,P27.0,P87.0,P40.0,P43.0,P21.0,P52.0,P9.0,0
3,P77.0,P35.0,P13.0,P21.0,P72.0,P96.0,P30.0,0
4,P85.0,P58.0,P5.0,P58.0,P30.0,P35.0,P76.0,0
...,...,...,...,...,...,...,...,...
3211,P5.0,P93.0,P99.0,P92.0,P40.0,P39.0,P50.0,0
3212,P43.0,P62.0,P73.0,P35.0,P82.0,P22.0,P82.0,0
3213,P28.0,P70.0,P38.0,P31.0,P83.0,P72.0,P18.0,0
3214,P86.0,P4.0,P6.0,P61.0,P79.0,P96.0,P36.0,0


In [41]:
print(myMode.shape)
print(ZERO.shape)

(3216, 8)
(3216, 8)


In [42]:
percentiles = [*range(0,101, 1)]
mapping_data = {}

columns_to_impute = banana.columns.difference(['Quality'])
for variable in columns_to_impute:
    mapping_data[variable] = [banana[banana['Quality']==0][variable].quantile(p / 100) for p in percentiles]

# Create the mapping DataFrame
mapping_df = pd.DataFrame(mapping_data, index=[f'P{p}' for p in percentiles])
print("Mapping DataFrame:")
mapping_df

Mapping DataFrame:


,Acidity,HarvestTime,Ripeness,Size,Softness,Sweetness,Weight
P0,-6.292339,-7.570008,-7.423155,-7.998074,-6.959320,-5.893079,-8.283002
P1,-4.608811,-5.382910,-5.035012,-5.497982,-4.860846,-4.627055,-5.330116
P2,-4.237076,-5.077713,-4.368813,-5.152749,-4.377747,-4.332015,-4.957582
P3,-3.918218,-4.702864,-4.077861,-4.804678,-4.064465,-4.071587,-4.708460
P4,-3.671925,-4.543398,-3.851930,-4.571162,-3.821002,-3.863941,-4.532867
...,...,...,...,...,...,...,...
P96,3.983738,1.496725,3.924699,1.667341,2.468967,0.866747,1.464498
P97,4.257724,1.661803,4.144970,1.921859,2.585758,1.109378,1.676883
P98,4.489732,1.995723,4.507051,2.229927,2.731268,1.326730,1.975377
P99,5.031205,2.524296,4.942510,2.737843,2.993727,1.719955,2.577319


In [43]:
def convert_percentiles_to_values(mdf, mapping_df):
    columns_to_impute = mdf.columns.difference(['Quality'])

    for column in columns_to_impute:
        mdf[column] = mdf[column].apply(lambda x: mapping_df.loc[x, column] if isinstance(x, str) and x.startswith('P') else x)
    return mdf

In [44]:
def path_to_revert(now, then):
    now = now.where(then.isna(), then)
    now = now.replace(r'^(P\d+)\.0$', r'\1', regex=True)
    return now

In [45]:
a = path_to_revert(myMode, ZERO)
a = convert_percentiles_to_values(a, mapping_df)

In [47]:
percent = pd.concat([a, df_resampled2[df_resampled2['Quality']==1][0:3216]], ignore_index=True)


In [49]:
def evaluate_oversampling2(X, y, classifier):

    # Train-test split on resampled data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Train the classifier
    classifier.fit(X_train, y_train)
    
    # Make predictions
    y_pred = classifier.predict(xtest)
    
    # Evaluate metrics
    accuracy = accuracy_score(ytest, y_pred)
    precision = precision_score(ytest, y_pred, average='weighted')
    recall = recall_score(ytest, y_pred, average='weighted')
    f1 = f1_score(ytest, y_pred, average='weighted')
    
    return accuracy, precision, recall, f1

In [50]:
classifiers = {
    "GaussianNaiveBayes": GaussianNB(),
    "KNN": KNeighborsClassifier(),
    "RandomForest": RandomForestClassifier(),
}

Percentile_Results = []

for name, classifier in classifiers.items():
    accuracy, precision, recall, f1 = evaluate_oversampling2(
        percent[['Size', 'Weight', 'Sweetness', 'Softness', 'HarvestTime', 'Ripeness',
       'Acidity']],
        percent[['Quality']],
        classifier
    )
    Percentile_Results.append({
        "Classifier": name,
        "Accuracy": accuracy,
        "Precision": precision,
        "Recall": recall,
        "F1 Score": f1
    })

Percentile_Results = pd.DataFrame(Percentile_Results)

c:\ProgramData\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\ProgramData\anaconda3\Lib\site-packages\sklearn\neighbors\_classification.py:238: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
c:\ProgramData\anaconda3\Lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


In [51]:
print('Percentile\n',Percentile_Results,'\n')
print('GaussianNaiveBayes\n',resultsNB,'\n')
print('KNN\n',resultsKNN,'\n')
print('RandomForest\n',resultsRF)

# Percentile
#             Classifier  Accuracy  Precision    Recall  F1 Score
# 0  GaussianNaiveBayes  0.885577   0.892009  0.885577  0.874980
# 1                 KNN  0.954808   0.955005  0.954808  0.953861
# 2        RandomForest  0.892308   0.905769  0.892308  0.880821 

# GaussianNaiveBayes
#                Method  Accuracy  Precision    Recall  F1 Score
# 0  RandomOverSampler  0.898077   0.909057  0.898077  0.901027
# 1              SMOTE  0.891346   0.904722  0.891346  0.894836
# 2             ADASYN  0.822115   0.870233  0.822115  0.832295
# 3    BorderlineSMOTE  0.808654   0.860595  0.808654  0.819859
# 4           SVMSMOTE  0.814423   0.869951  0.814423  0.825596 

# KNN
#                Method  Accuracy  Precision    Recall  F1 Score
# 0  RandomOverSampler  0.971154   0.973021  0.971154  0.971543
# 1              SMOTE  0.972115   0.973590  0.972115  0.972443
# 2             ADASYN  0.954808   0.960489  0.954808  0.955876
# 3    BorderlineSMOTE  0.968269   0.970343  0.968269  0.968715
# 4           SVMSMOTE  0.966346   0.969053  0.966346  0.966894 

# RandomForest
#                Method  Accuracy  Precision    Recall  F1 Score
# 0  RandomOverSampler  0.972115   0.972481  0.972115  0.972241
# 1              SMOTE  0.972115   0.972785  0.972115  0.972310
# 2             ADASYN  0.960577   0.961823  0.960577  0.960948
# 3    BorderlineSMOTE  0.962500   0.963116  0.962500  0.962716
# 4           SVMSMOTE  0.964423   0.965827  0.964423  0.964800


Percentile
            Classifier  Accuracy  Precision    Recall  F1 Score
0  GaussianNaiveBayes  0.895192   0.906031  0.895192  0.898178
1                 KNN  0.974038   0.974854  0.974038  0.974251
2        RandomForest  0.989423   0.989446  0.989423  0.989373 

GaussianNaiveBayes
               Method  Accuracy  Precision    Recall  F1 Score
0  RandomOverSampler  0.897115   0.908427  0.897115  0.900141
1              SMOTE  0.892308   0.903570  0.892308  0.895425
2             ADASYN  0.823077   0.870651  0.823077  0.833160
3    BorderlineSMOTE  0.804808   0.858978  0.804808  0.816402
4           SVMSMOTE  0.802885   0.864307  0.802885  0.815117 

KNN
               Method  Accuracy  Precision    Recall  F1 Score
0  RandomOverSampler  0.968269   0.970623  0.968269  0.968751
1              SMOTE  0.971154   0.972534  0.971154  0.971476
2             ADASYN  0.956731   0.961519  0.956731  0.957666
3    BorderlineSMOTE  0.966346   0.968754  0.966346  0.966857
4           SVMSMOTE  0.9